In [1]:
import time
import pandas as pd
 

from bs4 import BeautifulSoup
import urllib.request as req

import requests
import os
import datetime
import re
import glob

from team_index import get_teams

now = datetime.datetime.now()

os.chdir('/Users/sasno/Desktop/MyPandas/vleague')

headers = {"User-Agent": "Mozilla/5.0"}

# print('ディビジョンを選択してください:{v1_m, v2_m, v3_m, v1_w, v2_w}')
# division = input()
division = 'v1_m'
teams = get_teams(division)

if not os.path.isdir(division):
    os.makedirs(division)
os.chdir(division)

if not os.path.isdir('all'):
    os.makedirs('all')
os.chdir('all')

career_dir = 'career'
# print(career_dir)
if not os.path.isdir(career_dir):
    os.makedirs(career_dir)
    os.chdir(career_dir)

# print(os.getcwd())

career_lists = []

for team_id, team_name in teams.items():
    url = 'https://www.vleague.jp/record/team_players/{}'.format(team_id)
    # print(url)
    request = req.Request(url, headers=headers)
    response = req.urlopen(request)
    parse_html = BeautifulSoup(response, 'html.parser')

    tables = parse_html.find_all('table')[0]
    tr = tables.find_all('tr')
    a = tables.find_all('a')
    player_urls = []
    for i in a:
        href = i.attrs['href']
        # print(href)
        player_url = 'https://www.vleague.jp{}'.format(href)
        # print(player_url)
        player_urls.append(player_url)
    
    number_list = []
    player_list = []
    for i in tr[1:]:
        td = i.find_all('td')
        number = td[0].text
        number_list.append(number)
        player = td[1].text.replace('\xa0', '')
        player_list.append(player)
    # print(number_list, player_list)


    # print(os.getcwd())

    for i in range(len(player_list)):
        html = requests.get(player_urls[i], headers=headers)
        request_2 = req.Request(player_urls[i], headers=headers)
        response_2 = req.urlopen(request_2)
        parse_html_2 = BeautifulSoup(response_2, 'html.parser')
        table_2 = parse_html_2.find_all('table')

        if len(table_2) > 0: 
            data = pd.read_html(html.text, header = [1])
            stats = pd.DataFrame(data[0])
            # print(stats)
            stats = stats.rename({"チ｜ム": "チーム", "サ｜ブ": "サーブ", "ノ｜タッチ": "ノータッチ", "エ｜ス": "エース"}, axis=1)
            stats = stats.rename(columns={'打数':'アタック打数', '得点':'アタック得点', 
                            '失点':'アタック失点', '決定率':'アタック決定率',
                            'セ平ット均':'アタックセット平均','打数.1':'バックアタック打数', '得点.1':'バックアタック得点',
                            '失点.1':'バックアタック失点', '決定率.1':'バックアタック決定率', 
                            'セット平均':'アタック得点セット平均' ,'得点.2':'ブロック得点', 
                            'セット平均.1':'ブロック得点セット平均','セ平ット均.1':'ブロックセット平均', 
                            '打数.2':'サーブ打数', '得点.3':'サーブ得点', '失点.2':'サーブ失点',
                            '効果':'サーブ効果', '効果率':'サーブ効果率', '成功・優':'サーブレシーブ成功・優', 
                            '成功・良':'サーブレシーブ成功・良', '成功率':'サーブレシーブ成功率'})
            stats.insert(2, '選手名', player_list[i])
            stats.insert(9, 'アタック効果率', (((stats['アタック得点'] - stats['アタック失点']) / stats['アタック打数']) * 100).round(1))
            stats.insert(14, 'バックアタック効果率', (((stats['バックアタック得点'] - stats['バックアタック失点']) / stats['バックアタック打数']) * 100).round(1))
            career_lists.append(stats)
            career_database = pd.concat(career_lists, ignore_index=True)
            # csv = '{0}_{1}_{2:%Y}-{2:%m}-{2:%d}.csv'.format(number_list[i], player_list[i], now)
            # stats.to_csv(csv, index=False, encoding='cp932')
            # print('{}を作成'.format(csv))
career_database

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
1,レギュラーラウンド,ジェイテクトSTINGS,藤中優斗,27,70,129,53,8,41.1,34.9,...,37,4.5,224,93,67,56.5,53,13,5,71
2,セミファイナル（３位決定戦）,ジェイテクトSTINGS,藤中優斗,1,4,10,5,2,50.0,30.0,...,2,1.6,35,17,9,61.4,5,1,0,6
3,ファイナル,ジェイテクトSTINGS,藤中優斗,1,5,13,6,2,46.2,30.8,...,2,0.0,28,10,9,51.8,6,0,0,6
4,V.LEAGUE DIVISION1 MEN 2018-19,ジェイテクトSTINGS,藤中優斗,4,6,0,0,0,-,NaN,...,0,-,5,3,1,70,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,レギュラーラウンド,VC長野トライデンツ,長田翼,21,28,0,0,0,-,NaN,...,12,7.7,0,0,0,-,0,0,0,0
2962,Ｖ・チャレンジリーグⅡ男子 2015/16,VC長野トライデンツ,長田翼,3,5,0,0,0,-,NaN,...,1,3.6,0,0,0,-,0,0,0,0
2963,レギュラーラウンド,VC長野トライデンツ,長田翼,3,5,0,0,0,-,NaN,...,1,3.6,0,0,0,-,0,0,0,0
2964,レギュラーシーズン通算,NaN,長田翼,99,177,11,7,2,63.6,45.5,...,71,5.9,8,0,0,0.0,7,12,8,27


In [2]:
career_database = career_database.replace('-', None)
career_database

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
1,レギュラーラウンド,ジェイテクトSTINGS,藤中優斗,27,70,129,53,8,41.1,34.9,...,37,4.5,224,93,67,56.5,53,13,5,71
2,セミファイナル（３位決定戦）,ジェイテクトSTINGS,藤中優斗,1,4,10,5,2,50.0,30.0,...,2,1.6,35,17,9,61.4,5,1,0,6
3,ファイナル,ジェイテクトSTINGS,藤中優斗,1,5,13,6,2,46.2,30.8,...,2,0.0,28,10,9,51.8,6,0,0,6
4,V.LEAGUE DIVISION1 MEN 2018-19,ジェイテクトSTINGS,藤中優斗,4,6,0,0,0,46.2,NaN,...,0,0.0,5,3,1,70,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,レギュラーラウンド,VC長野トライデンツ,長田翼,21,28,0,0,0,80.0,NaN,...,12,7.7,0,0,0,0.0,0,0,0,0
2962,Ｖ・チャレンジリーグⅡ男子 2015/16,VC長野トライデンツ,長田翼,3,5,0,0,0,80.0,NaN,...,1,3.6,0,0,0,0.0,0,0,0,0
2963,レギュラーラウンド,VC長野トライデンツ,長田翼,3,5,0,0,0,80.0,NaN,...,1,3.6,0,0,0,0.0,0,0,0,0
2964,レギュラーシーズン通算,NaN,長田翼,99,177,11,7,2,63.6,45.5,...,71,5.9,8,0,0,0.0,7,12,8,27


In [113]:
regular19 = career_database[career_database['大会'] == '2019-20 V.LEAGUE DIVISION1 MEN']
regular19

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック効果率,...,サーブ効果,サーブ効果率,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,34.2,...,41,4.1,287,120,85,56.6,64,14,5,83
8,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,金丸晃大,27,51,17,8,1,47.1,41.2,...,8,4.3,2,0,1,25.0,8,5,0,13
60,2019-20 V.LEAGUE DIVISION1 MEN,東レアローズ,伏見大和,56,86,135,84,7,62.2,57.0,...,27,3.2,5,3,2,80.0,84,36,9,129
89,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,福山汰一,29,47,94,55,2,58.5,56.4,...,30,6.0,8,3,3,56.2,55,21,9,85
108,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,饒書涵,27,96,296,177,7,59.8,57.4,...,84,6.9,25,12,7,62,177,40,7,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,森崎健史,27,91,217,112,19,51.6,42.9,...,62,3.9,53,28,11,63.2,112,30,11,153
2932,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,松村文斗,24,3,10,1,2,10,-10.0,...,1,-10,21,8,7,54.8,1,0,0,1
2940,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,大沢健太郎,22,17,0,0,0,35.6,NaN,...,0,3,19,4,10,47.4,0,0,0,0
2946,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,高澤和貴,27,62,14,8,1,57.1,50.0,...,6,9.2,67,30,19,59,8,0,4,12


In [114]:
regular19['アタック決定率'] = regular19['アタック決定率'].astype(float)
regular19['バックアタック決定率'] = regular19['バックアタック決定率'].astype(float)
regular19['アタック得点セット平均'] = regular19['アタック得点セット平均'].astype(float)
regular19['ブロック得点セット平均'] = regular19['ブロック得点セット平均'].astype(float)
regular19['サーブ効果率'] = regular19['サーブ効果率'].astype(float)
regular19['サーブレシーブ成功率'] = regular19['サーブレシーブ成功率'].astype(float)

In [115]:
regular19.dtypes

大会              object
チーム             object
選手名             object
試合数              int64
セット数             int64
アタック打数           int64
アタック得点           int64
アタック失点           int64
アタック決定率        float64
アタック効果率        float64
バックアタック打数        int64
バックアタック得点        int64
バックアタック失点        int64
バックアタック決定率     float64
バックアタック効果率     float64
アタック得点セット平均    float64
ブロック得点           int64
ブロック得点セット平均    float64
サーブ打数            int64
ノータッチ            int64
エース              int64
サーブ失点            int64
サーブ効果            int64
サーブ効果率         float64
受数               int64
サーブレシーブ成功・優      int64
サーブレシーブ成功・良      int64
サーブレシーブ成功率     float64
アタック             int64
ブロック             int64
サーブ              int64
得点合計             int64
dtype: object

In [6]:
 ob_data = regular19[0:1].to_dict()
 print(ob_data['大会'][0])

2019-20 V.LEAGUE DIVISION1 MEN


In [116]:
attack_ave = (regular19['アタック得点'].sum() / regular19['アタック打数'].sum()) * 100
attack_ef = ((regular19['アタック得点'].sum() - regular19['アタック失点'].sum()) / regular19['アタック打数'].sum()) * 100
b_attack_ave = (regular19['バックアタック得点'].sum() / regular19['バックアタック打数'].sum()) * 100
b_attack_ef = ((regular19['バックアタック得点'].sum() - regular19['バックアタック失点'].sum()) \
        / regular19['バックアタック打数'].sum()) * 100
attack_by_set = regular19['アタック得点'].sum() / regular19['セット数'].sum()
block_by_set = regular19['ブロック得点'].sum() / regular19['セット数'].sum()
serve_ef = ((regular19['ノータッチ'].sum() + regular19['エース'].sum()) * 100 \
        + regular19['サーブ効果'].sum() * 25 - regular19['サーブ失点'].sum() * 25) \
        / regular19['サーブ打数'].sum()
s_receive_ave = \
     (regular19['サーブレシーブ成功・優'].sum() * 100 + \
     regular19['サーブレシーブ成功・良'].sum() * 50) \
     / regular19['受数'].sum()

In [159]:
# value = regular19.columns.values
# print('regular19.append(')
# for i in range(len(value)):
#     if i <= 2:
#         print("     ob_data['{}'][0],".format(value[i]))
#     else:
#         print("     regular19['{}'].sum(),".format(value[i]))
# print(')')

In [117]:
regular19.columns.values

array(['大会', 'チーム', '選手名', '試合数', 'セット数', 'アタック打数', 'アタック得点', 'アタック失点',
       'アタック決定率', 'アタック効果率', 'バックアタック打数', 'バックアタック得点', 'バックアタック失点',
       'バックアタック決定率', 'バックアタック効果率', 'アタック得点セット平均', 'ブロック得点', 'ブロック得点セット平均',
       'サーブ打数', 'ノータッチ', 'エース', 'サーブ失点', 'サーブ効果', 'サーブ効果率', '受数',
       'サーブレシーブ成功・優', 'サーブレシーブ成功・良', 'サーブレシーブ成功率', 'アタック', 'ブロック', 'サーブ',
       '得点合計'], dtype=object)

In [9]:
average_data = [
     ob_data['大会'][0],
     None,
     '合計',
     regular19['試合数'].sum(),
     regular19['セット数'].sum(),
     regular19['アタック打数'].sum(),
     regular19['アタック得点'].sum(),
     regular19['アタック失点'].sum(),
     attack_ave.round(1),
     attack_ef.round(1),
     regular19['バックアタック打数'].sum(),
     regular19['バックアタック得点'].sum(),
     regular19['バックアタック失点'].sum(),
     b_attack_ave.round(1),
     b_attack_ef.round(1),
     attack_by_set.round(1),
     regular19['ブロック得点'].sum(),
     block_by_set.round(1),
     regular19['サーブ打数'].sum(),
     regular19['ノータッチ'].sum(),
     regular19['エース'].sum(),
     regular19['サーブ失点'].sum(),
     regular19['サーブ効果'].sum(),
     serve_ef.round(1),
     regular19['受数'].sum(),
     regular19['サーブレシーブ成功・優'].sum(),
     regular19['サーブレシーブ成功・良'].sum(),
     s_receive_ave.round(1),
     regular19['アタック'].sum(),
     regular19['ブロック'].sum(),
     regular19['サーブ'].sum(),
     regular19['得点合計'].sum()
]

In [10]:
average = pd.DataFrame(columns=regular19.columns.values)
average.loc[0] = average_data
average['アタック決定率'][0]

49.2

In [22]:
regular19_s = regular19[regular19['バックアタック打数'] >= 100]
regular19_s = regular19_s[['チーム','選手名','バックアタック打数','バックアタック得点','バックアタック失点','バックアタック決定率','バックアタック効果率']]
regular19_s.sort_values(by=['バックアタック決定率'], ignore_index=True).iloc[::-1]

,チーム,選手名,バックアタック打数,バックアタック得点,バックアタック失点,バックアタック決定率,バックアタック効果率
10,堺ブレイザーズ,千々木駿介,126,81,9,64.3,57.1
9,サントリーサンバーズ,ムセルスキードミトリー,406,240,33,59.1,51.0
8,サントリーサンバーズ,季道帥,135,76,9,56.3,49.6
7,ジェイテクトSTINGS,西田有志,437,243,48,55.6,44.6
6,堺ブレイザーズ,樋口裕希,119,66,4,55.5,52.1
5,大分三好ヴァイセアドラー,バグナスブライアン,174,90,17,51.7,42.0
4,パナソニックパンサーズ,清水邦広,210,107,12,51.0,45.2
3,ウルフドッグス名古屋,高松卓矢,154,76,4,49.4,46.8
2,FC東京,プレモビッチピーター,444,215,29,48.4,41.9
1,大分三好ヴァイセアドラー,ヤカングマ,291,126,18,43.3,37.1


In [11]:
regular19_stand = regular19.loc[:,regular19.columns.str.contains('決定率|効果率|セット平均|成功率')].apply(lambda x : ((x - x.mean())*10/x.std()+50) ,axis=0)
div_value = regular19_stand.round(1)
div_value.columns.values

array(['アタック決定率', 'アタック効果率', 'バックアタック決定率', 'バックアタック効果率', 'アタック得点セット平均',
       'ブロック得点セット平均', 'サーブ効果率', 'サーブレシーブ成功率'], dtype=object)

In [121]:
regular19.insert(9,'アタック決定率偏差値',div_value['アタック決定率'])
regular19.insert(11,'アタック効果率偏差値',div_value['アタック効果率'])
regular19.insert(15,'バックアタック決定率偏差値',div_value['バックアタック決定率'])
regular19.insert(17,'バックアタック効果率偏差値',div_value['バックアタック効果率'])
regular19.insert(21,'アタック得点セット平均偏差値',div_value['アタック得点セット平均'])
regular19.insert(24,'ブロック得点セット平均偏差値',div_value['ブロック得点セット平均'])
regular19.insert(30,'サーブ効果率偏差値',div_value['サーブ効果率'])
regular19.insert(35,'サーブレシーブ成功率偏差値',div_value['サーブレシーブ成功率'])

In [123]:
# regular19 = career_database[career_database['大会'] == '2019-20 V.LEAGUE DIVISION1 MEN']
regular19

,大会,チーム,選手名,試合数,セット数,アタック打数,アタック得点,アタック失点,アタック決定率,アタック決定率偏差値,...,サーブ効果率偏差値,受数,サーブレシーブ成功・優,サーブレシーブ成功・良,サーブレシーブ成功率,サーブレシーブ成功率偏差値,アタック,ブロック,サーブ,得点合計
0,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,藤中優斗,29,79,152,64,12,42.1,46.5,...,49.2,287,120,85,56.6,55.1,64,14,5,83
8,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,金丸晃大,27,51,17,8,1,47.1,49.9,...,49.6,2,0,1,25.0,40.0,8,5,0,13
60,2019-20 V.LEAGUE DIVISION1 MEN,東レアローズ,伏見大和,56,86,135,84,7,62.2,60.2,...,47.6,5,3,2,80.0,66.3,84,36,9,129
89,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,福山汰一,29,47,94,55,2,58.5,57.7,...,52.6,8,3,3,56.2,54.9,55,21,9,85
108,2019-20 V.LEAGUE DIVISION1 MEN,ジェイテクトSTINGS,饒書涵,27,96,296,177,7,59.8,58.6,...,54.3,25,12,7,62.0,57.7,177,40,7,224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,森崎健史,27,91,217,112,19,51.6,53.0,...,48.9,53,28,11,63.2,58.3,112,30,11,153
2932,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,松村文斗,24,3,10,1,2,10.0,24.5,...,23.9,21,8,7,54.8,54.3,1,0,0,1
2940,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,大沢健太郎,22,17,0,0,0,35.6,42.0,...,47.3,19,4,10,47.4,50.7,0,0,0,0
2946,2019-20 V.LEAGUE DIVISION1 MEN,VC長野トライデンツ,高澤和貴,27,62,14,8,1,57.1,56.7,...,58.4,67,30,19,59.0,56.3,8,0,4,12


In [124]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline
import plotly.express as px

In [125]:
fig = px.line_polar(regular19.loc[:,div_ex], theta=regular19.loc[:,div_ex].columns.values, r=regular19.loc[:,div_ex].loc[240].values, range_r=[30,70], line_close=True)

In [126]:
fig.show()

In [129]:
div_ex = regular19.columns.str.contains('偏差値')
div_ex

array([False, False, False, False, False, False, False, False, False,
        True, False,  True, False, False, False,  True, False,  True,
       False, False, False,  True, False, False,  True, False, False,
       False, False, False,  True, False, False, False, False,  True,
       False, False, False, False])

In [137]:
div = regular19.loc[:,div_ex]
div.insert(0,'選手名',regular19['選手名'])
div = div.set_index('選手名')
div

,アタック決定率偏差値,アタック効果率偏差値,バックアタック決定率偏差値,バックアタック効果率偏差値,アタック得点セット平均偏差値,ブロック得点セット平均偏差値,サーブ効果率偏差値,サーブレシーブ成功率偏差値
選手名,,,,,,,,
藤中優斗,46.5,46.4,55.1,57.1,45.8,45.5,49.2,55.1
金丸晃大,49.9,49.7,55.1,NaN,40.4,40.7,49.6,40.0
伏見大和,60.2,57.4,75.8,75.4,47.2,60.0,47.6,66.3
福山汰一,57.7,57.1,75.8,NaN,48.7,61.8,52.6,54.9
饒書涵,58.6,57.5,75.8,NaN,54.3,60.0,54.3,57.7
...,...,...,...,...,...,...,...,...
森崎健史,53.0,50.6,34.5,NaN,49.2,54.5,48.9,58.3
松村文斗,24.5,25.1,49.4,NaN,41.8,58.2,23.9,54.3
大沢健太郎,42.0,NaN,39.6,NaN,49.6,43.7,47.3,50.7


In [131]:
regular19_stack =  div.stack().rename_axis(['選手名','項目']).reset_index().rename(columns={0:'偏差値'})
regular19_stack

,選手名,項目,偏差値
0,藤中優斗,アタック決定率偏差値,46.5
1,藤中優斗,アタック効果率偏差値,46.4
2,藤中優斗,バックアタック決定率偏差値,55.1
3,藤中優斗,バックアタック効果率偏差値,57.1
4,藤中優斗,アタック得点セット平均偏差値,45.8
...,...,...,...
1107,長田翼,バックアタック決定率偏差値,34.5
1108,長田翼,アタック得点セット平均偏差値,44.9
1109,長田翼,ブロック得点セット平均偏差値,40.1
1110,長田翼,サーブ効果率偏差値,55.2


In [132]:
reg_len = len(regular19_stack)
value = ['アタック決定率偏差値', 'アタック効果率偏差値', 'バックアタック決定率偏差値', 'バックアタック効果率偏差値', 'アタック得点セット平均偏差値', 'ブロック得点セット平均偏差値', 'サーブ効果率偏差値', 'サーブレシーブ成功率偏差値']
for i in range(0, 8):
    regular19_stack.loc[reg_len+i] = {'選手名':'平均','項目':value[i], '偏差値':50.0}

In [133]:
regular19_stack

,選手名,項目,偏差値
0,藤中優斗,アタック決定率偏差値,46.5
1,藤中優斗,アタック効果率偏差値,46.4
2,藤中優斗,バックアタック決定率偏差値,55.1
3,藤中優斗,バックアタック効果率偏差値,57.1
4,藤中優斗,アタック得点セット平均偏差値,45.8
...,...,...,...
1115,平均,バックアタック効果率偏差値,50.0
1116,平均,アタック得点セット平均偏差値,50.0
1117,平均,ブロック得点セット平均偏差値,50.0
1118,平均,サーブ効果率偏差値,50.0


In [135]:
player = regular19_stack[regular19_stack['選手名'].str.contains('西田有志|平均')]
player

,選手名,項目,偏差値
73,西田有志,アタック決定率偏差値,56.1
74,西田有志,アタック効果率偏差値,53.0
75,西田有志,バックアタック決定率偏差値,57.4
76,西田有志,バックアタック効果率偏差値,55.2
77,西田有志,アタック得点セット平均偏差値,83.4
78,西田有志,ブロック得点セット平均偏差値,62.4
79,西田有志,サーブ効果率偏差値,78.7
80,西田有志,サーブレシーブ成功率偏差値,45.1
1112,平均,アタック決定率偏差値,50.0
1113,平均,アタック効果率偏差値,50.0


In [136]:
fig = px.line_polar(player, r="偏差値", range_r=[30,70], theta="項目", color="選手名", line_close=True) 
fig.show()